In [1]:
# Add project root to Python path
import sys
import os
from pathlib import Path

# Find project root from current notebook location
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # When running from examples folder
    project_root = current_dir.parent.parent
else:
    # When running from project root
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import required modules
from typing import List, Dict, Any, Optional
from datetime import datetime
import polars as pl

from quantbt import (
    # Multi-timeframe strategy system
    MultiTimeframeTradingStrategy,
    BacktestEngine,
    
    # Basic modules
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # Order related
    Order, OrderSide, OrderType,
)

In [2]:
# Add project root to Python path
import sys
import os
from pathlib import Path

# Find project root from current notebook location
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # When running from examples folder
    project_root = current_dir.parent.parent
else:
    # When running from project root
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import required modules
from typing import List, Dict, Any, Optional
from datetime import datetime
import polars as pl

from quantbt import (
    # Multi-timeframe strategy system
    MultiTimeframeTradingStrategy,
    BacktestEngine,
    
    # Basic modules
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # Order related
    Order, OrderSide, OrderType,
)


class MultiTimeframeSMAStrategy(MultiTimeframeTradingStrategy):
    """
    Simplified multi-timeframe SMA strategy
    
    15-minute bars: Generate trading signals using short-term SMA crossover
    1-hour bars: Long-term trend filter
    
    Buy conditions:
    - 15m: Short SMA(10) > Long SMA(30)
    - 1h: Current price > Trend SMA(60)
    
    Sell conditions:
    - 15m: Short SMA(10) < Long SMA(30)
    """
    
    def __init__(self):
        timeframe_configs = {
            "15m": { "sma_windows": [10, 30] },
            "1h": { "sma_windows": [60] }
        }
        
        super().__init__(
            name="SimplifiedMultiTimeframeSMA",
            timeframe_configs=timeframe_configs,
            primary_timeframe="15m",  # Change primary timeframe
            position_size_pct=0.95,   # Increase position size
            max_positions=1
        )
        
    def _compute_indicators_for_symbol_and_timeframe(
        self, 
        symbol_data: pl.DataFrame, 
        timeframe: str, 
        config: Dict[str, Any]
    ) -> pl.DataFrame:
        """Calculate indicators per symbol + timeframe"""
        
        data = symbol_data.sort("timestamp")
        indicators = []
        
        if timeframe == "15m":
            # 15-minute bars: SMA for trading signals
            indicators.extend([
                pl.col("close").rolling_mean(config["sma_windows"][0]).alias("sma_short"),
                pl.col("close").rolling_mean(config["sma_windows"][1]).alias("sma_long")
            ])
            
        elif timeframe == "1h":
            # 1-hour bars: SMA for trend filtering
            indicators.append(
                pl.col("close").rolling_mean(config["sma_windows"][0]).alias("sma_trend")
            )
        
        return data.with_columns(indicators)
    
    def _analyze_multi_timeframe_signal(
        self, 
        data: Dict[str, Optional[Dict[str, Any]]], 
        symbol: str
    ) -> str:
        """Simplified cross-timeframe signal analysis"""
        
        d15m = data.get("15m", {})
        d1h = data.get("1h", {})
        
        # Extract essential indicator values
        price_15m = d15m.get('close')
        sma_short = d15m.get('sma_short')
        sma_long = d15m.get('sma_long')
        
        price_1h = d1h.get('close')
        sma_trend = d1h.get('sma_trend')

        # Hold if any essential indicator is missing
        if any(v is None for v in [price_15m, sma_short, sma_long, price_1h, sma_trend]):
            return "HOLD"
            
        has_position = symbol in self.get_current_positions()

        # Buy conditions
        is_uptrend = price_1h > sma_trend
        is_golden_cross = sma_short > sma_long
        
        if is_uptrend and is_golden_cross and not has_position:
            return "BUY"

        # Sell conditions
        is_dead_cross = sma_short < sma_long
        if is_dead_cross and has_position:
            return "SELL"
        
        return "HOLD"
    
    def generate_signals_multi_timeframe(
        self, 
        multi_current_data: Dict[str, Dict[str, Any]]
    ) -> List[Order]:
        """Generate multi-timeframe signals"""
        
        orders = []
        
        data_15m = multi_current_data.get("15m")
        if not data_15m:
            return orders
            
        symbol = data_15m.get('symbol')
        if not symbol:
            return orders
            
        signal = self._analyze_multi_timeframe_signal(multi_current_data, symbol)
        
        if signal == "BUY":
            orders.append(self._create_buy_order(symbol, data_15m))
        elif signal == "SELL":
            sell_order = self._create_sell_order(symbol)
            if sell_order:
                orders.append(sell_order)
        
        return orders
    
    def _create_buy_order(self, symbol: str, data_15m: Dict[str, Any]) -> Order:
        """Create buy order"""
        current_price = data_15m.get('close', 0)
        portfolio_value = self.get_portfolio_value()
        quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
        
        return Order(
            symbol=symbol,
            side=OrderSide.BUY,
            quantity=quantity,
            order_type=OrderType.MARKET
        )
    
    def _create_sell_order(self, symbol: str) -> Optional[Order]:
        """Create sell order"""
        current_positions = self.get_current_positions()
        quantity = current_positions.get(symbol, 0)
        
        if quantity > 0:
            return Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=quantity,
                order_type=OrderType.MARKET
            )
        return None

In [3]:
# 1. Upbit data provider
print("🔄 Initializing data provider...")
upbit_provider = UpbitDataProvider()

# 2. Multi-timeframe backtesting configuration
config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31),
    timeframe="15m",                 # 🚀 Change primary timeframe
    initial_cash=10_000_000,
    commission_rate=0.001,
    slippage_rate=0.0005,
    save_portfolio_history=True
)

# 3. Multi-timeframe SMA strategy
print("⚡ Initializing multi-timeframe SMA strategy...")
strategy = MultiTimeframeSMAStrategy()

print(f"📊 Strategy name: {strategy.name}")
print(f"🕐 Used timeframes: {strategy.available_timeframes}")
print(f"📈 Multi-timeframe: {strategy.is_multi_timeframe_strategy}")
print(f"🎯 Primary timeframe: {strategy.primary_timeframe}")

# 4. Broker configuration
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

# 5. Multi-timeframe backtest engine
print("🚀 Initializing multi-timeframe backtest engine...")
engine = BacktestEngine()
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

🔄 Initializing data provider...
⚡ Initializing multi-timeframe SMA strategy...
📊 Strategy name: SimplifiedMultiTimeframeSMA
🕐 Used timeframes: ['15m', '1h']
📈 Multi-timeframe: True
🎯 Primary timeframe: 15m
🚀 Initializing multi-timeframe backtest engine...


In [4]:
# 6. Execute backtesting
print("\n" + "=" * 60)
print("🚀 Executing multi-timeframe backtesting...")
print("=" * 60)
print(f"📊 Symbol: {config.symbols}")
print(f"📅 Period: {config.start_date} ~ {config.end_date}")
print(f"⏰ Timeframes: {strategy.available_timeframes}")
print(f"💰 Initial capital: {config.initial_cash:,} KRW")
print(f"💸 Commission: {config.commission_rate*100:.1f}%")
print(f"📉 Slippage: {config.slippage_rate*100:.2f}%")

result = engine.run(config)


🚀 Executing multi-timeframe backtesting...
📊 Symbol: ['KRW-BTC']
📅 Period: 2024-01-01 00:00:00 ~ 2024-12-31 00:00:00
⏰ Timeframes: ['15m', '1h']
💰 Initial capital: 10,000,000 KRW
💸 Commission: 0.1%
📉 Slippage: 0.05%


멀티 타임프레임 백테스팅 진행: 100%|██████████| 34968/34968 [00:00<00:00]


In [5]:
result.print_summary()

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $12,097,528
Total Return    : 20.98%
Annual Return   : 20.99%
Volatility      : 2.84%
Sharpe Ratio    : 7.38
Calmar Ratio    : 0.82
Sortino Ratio   : 1.08
Max Drawdown    : 25.45%
Total Trades    : 972
Win Rate        : 35.0%
Profit Factor   : 1.12
Execution Time  : 0.34s
